In [1]:
import subprocess; FOLDER_PATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
import warnings; warnings.filterwarnings("ignore")
import sys, os; sys.path.append(os.path.dirname(f'{FOLDER_PATH}/utils')); sys.path.append(os.path.dirname(f'{FOLDER_PATH}/scripts'))

from utils.base_helpers import DataReader
from utils.tf_helpers import WindowGenerator, BaseTFModel
import tensorflow as tf
from tensorflow.keras import models, layers
import pandas as pd
import numpy as np
import random

2022-05-23 01:08:52.363717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 01:08:52.363734: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-23 01:08:59.020930: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-23 01:08:59.020956: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (algopoly): /proc/driver/nvidia/version does not exist


In [10]:
seed = 1994
os.environ['PYTHONHASHSEED'] = str(seed)
# os.environ["TF_DETERMINISTIC_OPS"] = str(seed)
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

input_width = 48
optimizer = "adam" 
loss = "mse"
n_layers = 2
activation = "relu"

shift = 0 if input_width == 48 else -12 
config={
    "window": {"label_width": 24, "input_width": input_width, "shift": shift},
    "compiler":{"patience": 10, "epochs": 20, "optimizer": optimizer, "loss": loss, "verbose": 1}
}

ratios = [0.56095, 0.082, 0.082, 0.08288, 0.0838, 0.0838]

50

In [23]:
class ConvLSTM(BaseTFModel):
    def add_model(self, model):
        # model.add(layers.ConvLSTM1D(8, 5, return_sequences=True, activation=activation, padding="same"))
        model.add(layers.ConvLSTM1D(16, 5, activation=activation, padding="same", return_sequences=True))
        # model.add(layers.Permute((2,1,3)))
        # model.add(layers.Reshape((window.number_of_plants, -1)))
        
        model.add(layers.Permute((2,3,1)))
        model.add(layers.Dense(24))
        model.add(layers.Reshape((window.number_of_plants, -1)))
        return model

train_ratio_, valid_ratio_, test_ratio_ = ratios[:3]
print("Starting")
data = DataReader(number_of_plants=90)
data.process(train_ratio=train_ratio_, valid_ratio=valid_ratio_, test_ratio=test_ratio_)
window = WindowGenerator(data=data, **config["window"])

model = ConvLSTM(window)   # model.start_wandb(name=f"MLP_Period{i+1}")
print(model.model.summary())
model.compile_and_fit(**config["compiler"])

model.predict()

model.fitted_model.evaluate(window.valid, verbose=1)
model.fitted_model.evaluate(window.test, verbose=1)

wmape_month, wmape = model.calculate_accuracy("test")


Starting
Train start and end dates:	 2019-01-26 03:00:00 	 2020-10-02 22:00:00
Validation start and end dates:	 2020-10-02 23:00:00 	 2020-12-31 23:00:00
Test start and end dates:	 2021-01-01 00:00:00 	 2021-03-31 23:00:00
Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 start------ (Permute)       (None, 48, 90, 17)        0         
                                                                 
 conv_lstm1d_32 (ConvLSTM1D)  (None, 48, 90, 16)       10624     
                                                                 
 permute_32 (Permute)        (None, 90, 16, 48)        0         
                                                                 
 dense_30 (Dense)            (None, 90, 16, 24)        1176      
                                                                 
 reshape_32 (Reshape)        (None, 90, 384)           0         
                            

KeyboardInterrupt: 